In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import seaborn as sns

**Problem statemet**  

You are provided a list of cities and their coordinates in cities.csv. You must create the shortest possible path that visits all the cities. Your submission file is simply the ordered list in which you visit each city. Paths have the following constraints:

Paths must start and end at the North Pole (CityId = 0)
You must visit every city exactly once
The distance between two paths is the 2D Euclidean distance, except...
Every 10th step (stepNumber % 10 == 0) is 10% more lengthy unless coming from a prime CityId.

In [ ]:
cities = pd.read_csv('../input/cities.csv')
cities.head()

In [ ]:
cities.shape

**Prime Number Defination**  

From wikipedia.  

A prime number (or a prime) is a natural number greater than 1 that cannot be formed by multiplying two smaller natural numbers. A natural number greater than 1 that is not prime is called a composite number. For example, 5 is prime because the only ways of writing it as a product, 1 × 5 or 5 × 1, involve 5 itself. However, 6 is composite because it is the product of two numbers (2 × 3) that are both smaller than 6. Primes are central in number theory because of the fundamental theorem of arithmetic: every natural number greater than 1 is either a prime itself or can be factorized as a product of primes that is unique up to their order.

In [ ]:
def is_prime(num):
    if num > 1:
        for i in np.arange(2, np.sqrt(num+1)) :
            if num % i == 0:
                return False
        
        return True
    
    return False

In [ ]:
cities['prime'] = cities['CityId'].apply(is_prime)

In [ ]:
sum(cities.prime)

There are 197,769 cities out of which 17,802 are primes. To never get a penalty of 10% more distance we need 19,777 prime cities. Where prime cities are short by 1,975 (rounded). So we have to pick our penalties wisely.

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
ax.scatter(x = cities.X, y = cities.Y, alpha = 0.5, s = 1)
ax.set_title('Cities chart. North Pole, prime and non prime cities', fontsize = 16)
ax.scatter(x = cities.X[0], y = cities.Y[0], c = 'r', s =12)
ax.scatter(x = cities[cities.prime].X, y = cities[cities.prime].Y, s = 1, c = 'purple', alpha = 0.3)
ax.annotate('North Pole', (cities.X[0], cities.Y[0]),fontsize =12)
ax.set_axis_off()

In [ ]:
fig, (ax0,ax1) = plt.subplots(ncols = 2, figsize = (15,7))

ax0.set_title('Cities chart. North Pole, non prime cities', fontsize = 16)
ax0.scatter(x = cities.X[0], y = cities.Y[0], c = 'r', s =12)
ax0.scatter(x = cities[cities.prime == False ].X, y = cities[cities.prime == False].Y, s = 1, c = 'green', alpha = 0.3)
ax0.annotate('North Pole', (cities.X[0], cities.Y[0]),fontsize =12)
ax0.set_axis_off()

ax1.scatter(x = cities[cities.prime].X, y = cities[cities.prime].Y, s = 1, c = 'purple', alpha = 0.3)
ax1.set_title('Cities chart. North Pole, prime cities', fontsize = 16)
ax1.annotate('North Pole', (cities.X[0], cities.Y[0]),fontsize =12)
ax1.set_axis_off()

Prime cities are all well distributed amongst non prime cities. To get a sense of the data and paths, start with naive method where start from north pole and travel to all points in the order of thier distance with the north pole

In [ ]:
all_dist = distance.cdist(cities[['X','Y']][:1],cities[['X','Y']],metric = 'euclidean')
cities['dist_from_np'] = all_dist.T
cities.head()

In [ ]:
benchmark = cities.sort_values(by = 'dist_from_np').reset_index(drop=True)
b1 = benchmark.copy()
benchmark.rename(columns = {'CityId':'Path'}, inplace = True)
benchmark = benchmark.filter(items = ['Path'],axis=1)
benchmark = benchmark.append(benchmark.iloc[0]).reset_index(drop = True)
benchmark.head() # to make it uploadable format

It is uploaded and not surprisingly it does not compare well with other submissions. Just join the cities in the order from above,as we can see below the plot is almost all covered. Intuitively the paths should rughly follow point density above. Line chart should follow the pattern roughly blank spaces above should be blank here also.

In [ ]:
sns.set(rc={'figure.figsize':(10,10)})
sns.lineplot(x = b1.X, y = b1.Y, alpha = 0.3, color = 'grey')

Time to implement algorithms for solving TSP.